In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# elon_tweets = Path("./TweetsElonMusk.csv")
doge = Path("./doge.csv")
sp500 = Path("./sp500.csv")
tsla = Path("./tsla.csv")
sentiment = Path("./sentiment_df.csv")

In [ ]:
# elon_tweet_df = pd.read_csv(elon_tweets)
doge_df = pd.read_csv(doge)
sp500_df = pd.read_csv(sp500)
tsla_df = pd.read_csv(tsla)
sentiment_df = pd.read_csv(sentiment)

In [ ]:
import hvplot.pandas
hvplot.extension("bokeh")
doge_df.hvplot(y="Close")

In [ ]:
pct_chgs = doge_df["Close"].pct_change()
neg = 0
pos = 0
for i in pct_chgs:
    if i < 0:
        neg += 1
    else:
        pos += 1
display(neg)
display(pos)

In [ ]:
# elon_tweet_df = elon_tweet_df.rename({"date": "Date"}, axis=1)

In [ ]:
sentiment_df = sentiment_df.rename({"date": "Date"}, axis=1)

In [ ]:
# merged_df = pd.merge(elon_tweet_df, doge_df, on="Date")

In [ ]:
merged_df = pd.merge(tsla_df, sentiment_df, on="Date")

In [ ]:
merged_df.columns

In [ ]:
merged_df = merged_df[["Date", "Close", "replies_count", "retweets_count", "likes_count", "score_tag", "confidence"]]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
sentiment_cats = ["score_tag", "confidence"]

In [ ]:
numeric_cats = ["Date", "Close", "replies_count", "retweets_count", "likes_count"]

In [ ]:
enc = OneHotEncoder(sparse=False)
encoded_data = enc.fit_transform(merged_df[sentiment_cats])

In [ ]:
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names(sentiment_cats)
)

In [ ]:
encoded_df["Date"] = merged_df["Date"]
# encoded_df = encoded_df.groupby("Date").sum()

In [ ]:
encoded_df = encoded_df.groupby("Date").sum()

In [ ]:
encoded_df = encoded_df.reset_index()

In [ ]:
merged_df = merged_df[numeric_cats].groupby("Date").sum()

In [ ]:
merged_df = merged_df.reset_index()

In [ ]:
merged_df = pd.merge(encoded_df, merged_df[numeric_cats], on="Date")

In [ ]:
merged_df = merged_df.drop("Date", axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(merged_df)

In [ ]:
scaled_df = pd.DataFrame(
    scaled_data, 
    columns=merged_df.columns
)
merged_df = pd.concat([scaled_df, encoded_df], axis=1)

In [ ]:
merged_df["pct_chg"] = merged_df["Close"].pct_change()
merged_df = merged_df.drop("Close", axis=1)

In [ ]:
merged_df

In [ ]:
X = merged_df.iloc[1:, :-1]
y = merged_df.iloc[1:, -1]

In [ ]:
X = X.drop("Date", axis=1)

In [ ]:
# change to -1 for negative and 1 for positive
for i, v in y.items():
    if v > 0:
        y[i] = 1
    elif v < 0: 
        y[i] = -1
    else:
        y[i] = -1

In [ ]:
from sklearn.svm import SVC
# reg = LogisticRegression(random_state=1)
reg = SVC()
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
predictions = reg.predict(X_test)

In [ ]:
y_test

In [ ]:
y_test.value_counts()

In [ ]:
predictions

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)

# Fine Tuning Models

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV

In [ ]:
# parameters used for tuning

param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['sigmoid', 'rbf'],
              'degree': [1, 2, 3, 4]
             }

In [ ]:
#There are multiple methods to find the hyperparameters
#Method 1: GridSearch
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2)
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)
tuning_predictions1 = grid.predict(X_test)
print(classification_report(y_test, tuning_predictions1))

In [ ]:

#Method 2: HalvingGridSearch
grid2 = HalvingGridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid2.fit(X_train, y_train)

In [ ]:
print(grid2.best_params_)
tuning_predictions2 = grid2.predict(X_test)
print(classification_report(y_test, tuning_predictions2))

In [ ]:
#Method 3: RandomizedSearch
grid3 = RandomizedSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid3.fit(X_train, y_train)

In [ ]:
print(grid3.best_params_)
tuning_predictions3 = grid3.predict(X_test)
print(classification_report(y_test, tuning_predictions3))

In [ ]:
#Method 4: HalvingRandomSearch
grid4 = HalvingRandomSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid4.fit(X_train, y_train)

In [ ]:
print(grid4.best_params_)
tuning_predictions4 = grid4.predict(X_test)
print(classification_report(y_test, tuning_predictions4))